In [1]:
import veloxchem as vlx

In [2]:
h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)

nocc = molecule.number_of_alpha_electrons()
nvirt = basis.get_dimension_of_basis() - nocc

print("Number of occupied orbitals:", nocc)
print("Number of unoccupied orbitals:", nvirt)

Number of occupied orbitals: 5
Number of unoccupied orbitals: 19


In [3]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

In [5]:
E = scf_results["E_alpha"]

In [6]:
erimo_drv = vlx.MOIntegralsDriver()

ovov = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_ovov")
print("(ov|ov):", ovov.shape)

(ov|ov): (5, 19, 5, 19)


In [7]:
e_mp2_ss = 0.0
e_mp2_os = 0.0

# extract the occupied subset of the orbital energies
e_o = E[:nocc]
# extract the virtual subset of the orbital energies
e_v = E[nocc:]

for i in range(nocc):
    for j in range(nocc):
        for a in range(nvirt):
            for b in range(nvirt):
                # enegy denominators
                e_ijab = e_v[a] + e_v[b] - e_o[i] - e_o[j]

                # update opposite-spin component of the energy
                e_mp2_os -= (ovov[i, a, j, b] * ovov[i, a, j, b]) / e_ijab

                # update same-spin component of the energy
                e_mp2_ss -= (
                    ovov[i, a, j, b] * (ovov[i, a, j, b] - ovov[i, b, j, a]) / e_ijab
                )

In [8]:
print(f"Opposite-spin MP2 energy: {e_mp2_os:12.8f}")
print(f"Same-spin MP2 energy    : {e_mp2_ss:12.8f}")
print(f"MP2 energy              : {e_mp2_os + e_mp2_ss:12.8f}")

Opposite-spin MP2 energy:  -0.15163083
Same-spin MP2 energy    :  -0.05138187
MP2 energy              :  -0.20301270


In [9]:
mp2_drv = vlx.Mp2Driver()
mp2_drv.ostream.mute()

mp2_results = mp2_drv.compute_conventional(molecule, basis, scf_drv.mol_orbs)

In [10]:
print(f"Energy difference: {mp2_results['mp2_energy']:12.8f}")

Energy difference:  -0.20301270


In [11]:
dimer_xyz = """6
2 water 100 Å apart                                                                                                            
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
O  100.000000000000        0.000000000000        0.000000000000                         
H  100.000000000000        0.740848095288        0.582094932012                         
H  100.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_xyz_string(dimer_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)

In [12]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
scf_results = scf_drv.compute(molecule, basis)

mp2_dimer_results = mp2_drv.compute_conventional(molecule, basis, scf_drv.mol_orbs)

In [14]:
print("=" * 30)
print("MP2 energies")
print("-" * 30)
print(f"Dimer       : {mp2_dimer_results['mp2_energy']:16.10f}")
print(f"Two monomers: {2 * mp2_results['mp2_energy']:16.10f}")
print("=" * 30)

MP2 energies
------------------------------
Dimer       :    -0.4060254140
Two monomers:    -0.4060254051


In [15]:
molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31G", ostream=None)

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

In [16]:
# MO coefficients
C = scf_results["C_alpha"]

# orbital energies
E = scf_results["E_alpha"]

In [17]:
import numpy as np

F = np.diag(E)

# Get the core Hamiltonian in MO basis
T_ao = vlx.compute_kinetic_energy_integrals(molecule, basis)

V_nucpot_ao = vlx.compute_nuclear_potential_integrals(molecule, basis)

h = np.einsum("ai, ab, bj -> ij", C, T_ao + V_nucpot_ao, C)

# Compute the 2-electron integrals
fock_drv = vlx.FockDriver()
g_ao = fock_drv.compute_eri(molecule, basis)

g_1 = np.einsum("ds, abcd -> abcs", C, g_ao)
g_2 = np.einsum("cr, abcs -> abrs", C, g_1)
g_3 = np.einsum("bq, abrs -> aqrs", C, g_2)

g = np.einsum("ap, aqrs -> pqrs", C, g_3)

In [18]:
H0 = F

V_1 = h - F
V_2 = g

In [19]:
import multipsi as mtp

ci_drv = mtp.CIDriver()
ci_drv.ostream.mute()

space = mtp.OrbSpace(molecule, scf_drv.mol_orbs)
space.fci()

In [20]:
E_FCI = []

for lambda_val in [-0.01, 0, 0.01]:

    ci_drv._update_integrals(
        molecule.nuclear_repulsion_energy(), H0 + lambda_val * V_1, lambda_val * V_2
    )

    ci_results = ci_drv.compute(molecule, basis, space, 1)

    E_FCI.append(ci_results["energies"][0])

In [21]:
E_MP2 = 0.5 * (E_FCI[0] + E_FCI[2] - 2 * E_FCI[1]) / 0.01**2

In [22]:
print(f"MP2 energy correction:{E_MP2:12.8f}")

MP2 energy correction: -0.12747116


In [23]:
mp2_results = mp2_drv.compute(molecule, basis, scf_drv.mol_orbs)

In [24]:
print(f"MP2 energy correction:{mp2_results['mp2_energy']:12.8f}")

MP2 energy correction: -0.12747066
